In [1]:
import os
import pandas as pd
from tqdm.notebook import tqdm
import git
import pdb
import sys
import copy

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")
uncompressed = os.path.join(git_root, "annotated/uncompressed/")

# Column names based on annotator GitHub
columns = ["chromosome", "start", "stop", "name", "intensity", "strand", "gene_id", "gene_name", "genic_region_type", "all_overlapping_annotation"]

In [3]:
dirs = {
    'spidr': {
        # 'peaks_switched_strands_filtered': os.path.join(uncompressed, "spidr/peaks_switched_strands_filtered"),
        # 'miRNAadj': os.path.join(uncompressed, "spidr/spidr_annotated_bed_with_miRNAadj"),
        'miRNAadj': os.path.join(uncompressed, "spidr/spidr_annotated_bed_with_miRNAadj")
    },
    'encode': {
        'miRNAadj': os.path.join(uncompressed, "encode/encode_annotated_bed_with_miRNAadj"),
        'downsampled_miRNAadj': os.path.join(uncompressed, "encode/downsampled_encode_annotated_bed_with_miRNAadj"),
        # 'peaks_filtered': os.path.join(uncompressed, "encode/peaks_filtered"),
        # 'downsampled_peaks_filtered': os.path.join(uncompressed, "encode/downsampled_peaks_filtered"),
    }
}

# TODO: Figure out how to use traverse here
files = {}
for data_type in dirs.keys():
    files[data_type] = {}
    for key in dirs[data_type].keys():
        dir = dirs[data_type][key]
        files[data_type][key] = [os.path.join(dir, file) for file in os.listdir(dir)]

In [4]:
def get_percentages(annotated_files, cols, percent_by='sum'):
    percents = []
    
    for file in tqdm(annotated_files, total=len(annotated_files)):
        # Check if file is empty
        try:
            with open(file, 'r') as f:
                if len(f.read()) == 0:
                    continue
        except UnicodeDecodeError:
            print(file)
            sys.exit(0)

        # Read in each file as a dataframe
        df = pd.read_csv(file, sep="\t", names=cols)
        basename = os.path.basename(file).replace('.txt', '')
        subset = df[["genic_region_type", "intensity"]]

        if percent_by == 'sum':
            intensities = subset.groupby(by=["genic_region_type"]).sum('intensity')
        elif percent_by == 'count':
            intensities = subset.groupby(by=["genic_region_type"]).count()

        intensities.columns = [f"{basename}"]
        total = intensities.sum().values.item()
        percent_col = (intensities / total) * 100
        percents.append(percent_col)


    percents_df = pd.concat(percents, axis=1, join='outer')
    percents_df.fillna(value=0, inplace=True)

    return percents_df

In [5]:
def rename_columns(df):
    tmp = df.copy()
    new_tmp_cols = []

    for col in tmp.columns:
        # Keep Bethyl and CST separately
        if "Bethyl" in col or "CST" in col:
            new_tmp_cols.append("_".join(col.split("_")[0:2]))
        else:
            new_col = col.split("_")[0]
            if "rep1" in new_col:
                new_col = new_col.replace("rep1", "")
            new_tmp_cols.append(new_col)

    tmp.columns = new_tmp_cols
    return new_tmp_cols, tmp

In [6]:
def create_percentage_tables(dict):
    pass

In [7]:
spidr_df = get_percentages(files['spidr']['miRNAadj'], cols=columns, percent_by='sum')
spidr_cols, spidr_df = rename_columns(spidr_df)

encode_df = get_percentages(files['encode']['miRNAadj'], cols=columns, percent_by='sum')
encode_cols, encode_df = rename_columns(encode_df)

encode_downsampled_df = get_percentages(files['encode']['downsampled_miRNAadj'], cols=columns, percent_by='sum')
encode_downsampled_cols, encode_downsampled_df = rename_columns(encode_downsampled_df)

  0%|          | 0/58 [00:00<?, ?it/s]

  0%|          | 0/43 [00:00<?, ?it/s]

  0%|          | 0/41 [00:00<?, ?it/s]

## Encode Percent Counts by Region Type from Original Paper

In [8]:
# Read the excel file, skipping the first row to ensure proper column names
encode_supp_path = os.path.join(git_root, "annotated/Summary_info_encode_Suppl_Data_4.xlsx")
encode_supp_data = pd.read_excel(encode_supp_path, skiprows=1)

# Filtering for only 'K562' cell lines
encode_supp_data = encode_supp_data[encode_supp_data['Cell Line'] == 'K562']

# Get the list of gene symbols
gene_symb = encode_supp_data[['Official Gene Symbol']]

# Get all columns corresponding to total counts and counts of subsets (e.g. CDS, miRNA, etc)
region_counts = encode_supp_data[encode_supp_data.columns[-17:].tolist()]

# Merge the dataframes by index
raw_counts = gene_symb.join(region_counts)
raw_counts.set_index('Official Gene Symbol', inplace=True)
raw_counts.to_csv(os.path.join(git_root, "output", "encode_supp_raw_counts_by_region.csv"))

# Divide subset counts by total
percent_counts = raw_counts[raw_counts.columns[1:]].div(raw_counts['IDR peak #'], axis=0) * 100

# Transpose the dataframe so it's in the same shape as encode and spidr along with corresponding columns
encode_supp = percent_counts.T

# Manually renaming things in supplementary data to match annotator output
supp_to_annot = {
    "distintron": "distintron500",
    "noncoding_distintron": "distnoncoding_intron500",
    "proxintron": "proxintron500",
    "noncoding_proxintron": "proxnoncoding_intron500",
    # "miRNA_proximal": "miRNA_adjacent"
}

new_index = {}

for idx in encode_supp.index:
    if idx in supp_to_annot:
        new_index[idx] = supp_to_annot[idx]
    else:
        new_index[idx] = idx

encode_tmp = encode_supp.copy()

encode_tmp = encode_tmp.rename(index=new_index)

In [9]:
def augment_dfs(ref, dfs):
    # Find all the columns with antibody variants in the reference dataframe
    cols = [col for col in ref.columns if ("CST" in col) or ("Bethyl" in col)]
    
    for col in cols:
        for df in dfs:
            # If antibody variant isn't present, add it as a column
            if col not in df.columns:
                protein_name = col.split("_")[0]
                antibody = col.split("_")[1]
                df[f'{protein_name}_{antibody}'] = df[f'{protein_name}']

    return dfs

In [17]:
encode_df_aug, encode_downsampled_df_aug, encode_supp_aug = augment_dfs(spidr_df, [encode_df, encode_downsampled_df, encode_tmp])

dfs = {
    "spidr": spidr_df,
    "encode": encode_df_aug,
    "encode_downsampled": encode_downsampled_df_aug,
    "encode_supp": encode_supp_aug
}

final = {key: None for key in dfs.keys()}

In [18]:
def get_common(dataframe_dict):
    cols = []
    indices = []
    
    for df in dataframe_dict.values():
        cols.append(set(df.columns))
        indices.append(set(df.index))
    
    common_cols = list(cols[0].intersection(*cols[1:]))
    common_indices = list(indices[0].intersection(*indices[1:]))
    return common_cols, common_indices

In [19]:
common_cols, common_idx = get_common(dfs)
output_root = os.path.join(git_root, "output")

for key, df in dfs.items():
    # Keep common columns
    tmp = df.copy()
    tmp = tmp[common_cols]

    # Keep only region names that are common to all dataframes
    tmp = tmp.filter(items=common_idx, axis=0)
    
    # Remove duplicated columns
    tmp = tmp.loc[:, ~tmp.columns.duplicated()]

    # Save to disk
    final[key] = tmp
    final[key].to_csv(os.path.join(output_root, f"{key}_percent_by_region.csv"))